<a href="https://colab.research.google.com/github/areias/healthcare_fraud/blob/main/healthcare_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Healthcare Fraud Detection

https://www.kaggle.com/rohitrox/healthcare-provider-fraud-detection-analysis

first steps, data exploration and summary stats of data

i think its a binary classification problem, let's maybe try a logistic regression on it as a baseline 

create a schema to upload to neo4j as a graph database

try graph machine learning models on it 

experiment tracking https://wandb.ai/

imbalanced data
